In [ ]:
###### import random
import time
import requests
import random
from bs4 import BeautifulSoup
import json
import pandas as pd

# Set the seed immediately after importing random
random.seed(1)

data_consent_info = """DATA CONSENT INFORMATION:
Please read:
we wish to record your response data
to an anonymised public data repository.
Your data will be used for educational teaching purposes practising data analysis and visualisation.
Please type yes in the box below if you consent to the upload."""
print(data_consent_info)
result = input("> ")
if result == "yes" or "Yes":
    print("Thanks for your participation.")
    print("Please contact kwanhwi.lee.21@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")
else:
    # end code execution by raising an exception
    raise(Exception("User did not consent to continue test."))

id_instructions = """
Enter your anonymised ID
To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress
e.g. if your friend was called Charlie Brown and film star was Tom
Cruise
then your unique identifer would be CBTC """
print(id_instructions)
user_id = input("> ")
print("User entered id:", user_id)
print("Enter your age:")
age = input("> ")
print("Enter your gender(male or female):")
gender = input("> ")

def generate_question(difficulty):
    """Generates a math question based on the difficulty level."""
    operations = ['+', '-', '*']
    question = str(random.randint(1, 10))

    for _ in range(difficulty):
        operation = random.choice(operations)
        number = random.randint(1, 10)
        question += f' {operation} {number}'

    return question

def ask_question(question):
    """Displays a question and returns the user's answer and time taken."""
    start_time = time.time()
    try:
        answer = int(input(f"{question} = "))
    except ValueError:  # In case non-integer input is given
        answer = None
    end_time = time.time()
    return answer, end_time - start_time

def maths_ability_test():
    total_questions = 0
    correct_answers = 0
    total_time = 0
    start_test = time.time()

    while time.time() - start_test < 180:  # 180 seconds = 3 minutes
        difficulty = total_questions // 5 + 1  # Increase difficulty every 5 questions
        difficulty = min(difficulty, 5)  # Max difficulty is 5

        question = generate_question(difficulty)
        correct_answer = eval(question)

        # Ensuring the answer is an integer and within the specified range
        if not isinstance(correct_answer, int) or not (1 <= correct_answer <= 500):
            continue

        start_time = time.time()  # Set start time for each question
        print(f"Question {total_questions + 1}:")
        user_answer, time_taken = ask_question(question)

        # Check if the answer is correct and an integer
        if user_answer is not None and user_answer == correct_answer:
            correct_answers += 1

        total_questions += 1
        total_time += time_taken

        # Wait until 2 seconds have passed before displaying the next question
        time.sleep(max(0, 2 - time_taken))

    average_time = total_time / total_questions if total_questions > 0 else 0

    return correct_answers, total_questions, average_time

# Running the test
score, questions_attempted, average_time = maths_ability_test()
print(f"Test Completed. Score: {score}/{questions_attempted}, Average Time: {average_time:.2f} seconds/question")

# Define data_dict here
data_dict = {
    "entry.1576333334": user_id,  # Replace with the actual entry ID for User ID
    "entry.557558376": age,       # Replace with the actual entry ID for Age
    "entry.1913945443": gender,   # Replace with the actual entry ID for Gender
    "entry.2049298113": score,    # Replace with the actual entry ID for Score
    "entry.744027016": average_time  # Replace with the actual entry ID for Average Time
}

# Now create the DataFrame
dataframe = pd.DataFrame([data_dict])
myjson = dataframe.to_json()
newdf = pd.read_json(myjson)

# URL of the Google Form submission endpoint
forms_url = "https://docs.google.com/forms/d/e/1FAIpQLSc6cdMzDdptB9FlgPrdABZSccaRfeKEKaCFa52_63de4Z8QKA/formResponse"

# Send the data to the Google Form
response = requests.post(forms_url, data=data_dict)
if response.ok:
    print("Data submitted successfully.")
else:
    print("Failed to submit data")